In [22]:
import pandas as pd
import numpy as np
import pickle
import Bio   
import plotly.express as px
import plotly.graph_objects as go
from jproperties import Properties
import wandb
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, AutoModel, TrainingArguments, Trainer
from motif_utils import seq2kmer # Soruced from https://github.com/jerryji1993/DNABERT
import torch
from datasets import Dataset
import evaluate
import json
from load_data import load_data

In [23]:
train, val, test = create_dataset()

c:\Users\zeusg\Envs\real-fast\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [32]:
df = pd.read_csv("D:/ML/bert_dna_weight/data/capstone_body_weight_Statistical_effect_size_analysis_genotype_early_adult_scaled.csv")
df

,Unnamed: 0,procedure_name,parameter_name,allele_accession_id,gene_accession_id,gene_symbol,allele_symbol,zygosity,strain,est_f_ea,...,se_m_ea,p_m_ea,p_adj_m_ea,est_type_ea,add_info_ea,wt_f_ea_count,wt_m_ea_count,mut_f_ea_count,mut_m_ea_count,total_number_of_mutants
0,0,body weight,body weight,mgi:1857129,MGI:88057,Apoe,Apoe<tm1Unc>,homozygote,c57bl/6,-0.049546,...,0.122518,6.860893e-01,6.860893e-01,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",278.0,271.0,18.0,21.0,39.0
1,1,acoustic startle&ppi,body weight,mgi:1857236,MGI:97569,Abcb4,Abcb4<tm1Bor>,homozygote,balb/cbyj,0.367317,...,0.123266,3.670000e-10,3.670000e-10,Sex-specific genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",10.0,10.0,10.0,10.0,20.0
2,2,body weight,body weight,mgi:1857493,MGI:107160,Hmx3,Hmx3<tm1Ebo>,homozygote,stock hmx3<hx>,-0.611352,...,0.203740,1.680000e-07,3.370000e-07,Sex-specific genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",17.0,15.0,15.0,16.0,31.0
3,3,body weight,body weight,mgi:1928287,MGI:95820,Grin2a,Grin2a<tm1Rsp>,homozygote,c57bl/6jico;129p2,-0.068684,...,0.083171,4.119273e-01,4.119273e-01,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",27.0,23.0,15.0,15.0,30.0
4,4,body weight,body weight,mgi:2182460,MGI:1277959,Dlg4,Dlg4<tm1Grnt>,homozygote,c57bl/6jico;129p2,-0.549791,...,0.163915,1.220000e-08,1.220000e-08,Sex-specific genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",27.0,23.0,10.0,7.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5700,5700,body weight,body weight,not-released-fc11d38042,MGI:2443598,Maats1,Maats1<em1(IMPC)Mbp>,homozygote,c57bl/6ncrl,-0.031436,...,0.175926,8.581843e-01,8.581843e-01,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5083.0,5198.0,8.0,8.0,16.0
5701,5701,body weight,body weight,not-released-fc6d76bf2b,MGI:2663998,Plac9a,Plac9a<tm1b(EUCOMM)Hmgu>,homozygote,c57bl/6ntac,-0.002483,...,0.171023,9.884174e-01,9.884174e-01,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5807.0,5829.0,8.0,8.0,16.0
5702,5702,body weight,body weight,not-released-fe95f2c63e,MGI:1921364,Crot,Crot<em1(IMPC)KMPC>,homozygote,c57bl/6ntac,-0.845826,...,0.237108,8.270155e-01,8.270155e-01,Sex-specific genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5807.0,5829.0,8.0,7.0,15.0
5703,5703,body weight,body weight,not-released-ff96ec2653,MGI:1913998,Erich2,Erich2<em1(IMPC)Marc>,homozygote,c57bl/6n,-0.524457,...,0.197315,5.602648e-01,8.356967e-01,Sex-specific genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5724.0,5744.0,7.0,7.0,14.0


In [44]:
1 - len(df.gene_symbol.unique())/len(df)


0.03270284603146545

In [42]:
df[df["gene_symbol"].duplicated()].head(10)

,Unnamed: 0,procedure_name,parameter_name,allele_accession_id,gene_accession_id,gene_symbol,allele_symbol,zygosity,strain,est_f_ea,...,se_m_ea,p_m_ea,p_adj_m_ea,est_type_ea,add_info_ea,wt_f_ea_count,wt_m_ea_count,mut_f_ea_count,mut_m_ea_count,total_number_of_mutants
10,10,acoustic startle&ppi,body weight,mgi:3056485,MGI:108078,Sfrp2,Sfrp2<C50F>,homozygote,c57bl/6j,-0.989220,...,0.126649,9.950000e-14,9.950000e-14,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",180.0,139.0,11.0,10.0,21.0
14,14,acoustic startle&ppi,body weight,mgi:3621117,MGI:108078,Sfrp2,Sfrp2<tm1Aksh>,homozygote,c57bl/6j,-0.662875,...,0.148941,5.980000e-05,5.980000e-05,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",180.0,139.0,15.0,8.0,23.0
125,125,body weight,body weight,mgi:4364130,MGI:2385206,Pabpc4,Pabpc4<tm1e(KOMP)Wtsi>,homozygote,c57bl/6n,-0.638988,...,0.301560,3.411928e-02,3.411928e-02,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5724.0,5744.0,7.0,7.0,14.0
251,251,body weight,body weight,mgi:4431921,MGI:104510,Myo7a,Myo7a<tm1a(EUCOMM)Wtsi>,homozygote,c57bl/6n,-0.655075,...,0.188841,5.246410e-04,2.203907e-03,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5724.0,5744.0,7.0,8.0,15.0
471,471,acoustic startle&ppi,body weight,mgi:4441779,MGI:1919305,Slc38a10,Slc38a10<tm2a(EUCOMM)Wtsi>,homozygote,c57bl/6dnk,-1.036508,...,0.160374,7.280000e-10,7.280000e-10,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",119.0,89.0,17.0,18.0,35.0
635,635,body weight,body weight,mgi:4842622,MGI:1344351,Dlg2,Dlg2<tm1a(EUCOMM)Wtsi>,homozygote,c57bl/6n,-0.186159,...,0.180511,3.024288e-01,3.629146e-01,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5724.0,5744.0,8.0,8.0,16.0
738,738,body weight,body weight,mgi:5442804,MGI:2384936,Spns2,Spns2<tm1b(KOMP)Wtsi>,homozygote,b6brd;b6dnk;b6n-tyr<c-brd>,-0.266915,...,0.344705,1.447070e-04,8.682400e-04,Sex-specific genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",97.0,87.0,8.0,9.0,17.0
853,853,body weight,body weight,mgi:5501065,MGI:1913955,Dnase1l2,Dnase1l2<tm1.1(KOMP)Wtsi>,homozygote,c57bl/6ntac,-1.197042,...,0.183200,2.390000e-23,7.180000e-23,Sex-specific genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5807.0,5829.0,22.0,21.0,43.0
854,854,body weight,body weight,mgi:5501071,MGI:1336167,Prkab1,Prkab1<tm1b(KOMP)Wtsi>,homozygote,c57bl/6n,0.108506,...,0.119641,2.959190e-04,5.918380e-04,Sex-specific genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5724.0,5744.0,30.0,28.0,58.0
901,901,body weight,body weight,mgi:5513812,MGI:1920180,Ppp2r2b,Ppp2r2b<tm1b(KOMP)Wtsi>,homozygote,c57bl/6ncrl,-0.350291,...,0.188590,1.978075e-01,2.373690e-01,Sex-specific genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5083.0,5198.0,8.0,8.0,16.0


In [29]:
train

,Unnamed: 0,procedure_name,parameter_name,allele_accession_id,gene_accession_id,gene_symbol,allele_symbol,zygosity,strain,est_f_ea,...,p_m_ea,p_adj_m_ea,est_type_ea,add_info_ea,wt_f_ea_count,wt_m_ea_count,mut_f_ea_count,mut_m_ea_count,total_number_of_mutants,dna_seq
1011,1011,body weight,body weight,mgi:5548415,MGI:1274790,Alox12e,Alox12e<tm1b(EUCOMM)Wtsi>,homozygote,c57bl/6n,-0.056970,...,0.811250,0.995652,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5724.0,5744.0,9.0,7.0,16.0,TTTAAG TTAAGG TAAGGG AAGGGA AGGGAG GGGAGA GGAG...
1011,1011,body weight,body weight,mgi:5548415,MGI:1274790,Alox12e,Alox12e<tm1b(EUCOMM)Wtsi>,homozygote,c57bl/6n,-0.056970,...,0.811250,0.995652,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5724.0,5744.0,9.0,7.0,16.0,GGTGTC GTGTCT TGTCTT GTCTTC TCTTCC CTTCCC TTCC...
1011,1011,body weight,body weight,mgi:5548415,MGI:1274790,Alox12e,Alox12e<tm1b(EUCOMM)Wtsi>,homozygote,c57bl/6n,-0.056970,...,0.811250,0.995652,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5724.0,5744.0,9.0,7.0,16.0,TGTTCT GTTCTG TTCTGG TCTGGT CTGGTC TGGTCA GGTC...
1011,1011,body weight,body weight,mgi:5548415,MGI:1274790,Alox12e,Alox12e<tm1b(EUCOMM)Wtsi>,homozygote,c57bl/6n,-0.056970,...,0.811250,0.995652,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5724.0,5744.0,9.0,7.0,16.0,GATTGT ATTGTG TTGTGT TGTGTT GTGTTT TGTTTG GTTT...
1011,1011,body weight,body weight,mgi:5548415,MGI:1274790,Alox12e,Alox12e<tm1b(EUCOMM)Wtsi>,homozygote,c57bl/6n,-0.056970,...,0.811250,0.995652,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5724.0,5744.0,9.0,7.0,16.0,TTCACA TCACAT CACATC ACATCG CATCGG ATCGGT TCGG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3968,3968,body weight,body weight,mgi:6156505,MGI:1929266,Snd1,Snd1<em1(IMPC)Tcp>,homozygote,c57bl/6ncrl,-0.774071,...,0.016955,0.025432,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5083.0,5198.0,2.0,3.0,5.0,CTGTGC TGTGCA GTGCAC TGCACC GCACCC CACCCT ACCC...
3968,3968,body weight,body weight,mgi:6156505,MGI:1929266,Snd1,Snd1<em1(IMPC)Tcp>,homozygote,c57bl/6ncrl,-0.774071,...,0.016955,0.025432,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5083.0,5198.0,2.0,3.0,5.0,TGGCAA GGCAAG GCAAGT CAAGTC AAGTCC AGTCCA GTCC...
3968,3968,body weight,body weight,mgi:6156505,MGI:1929266,Snd1,Snd1<em1(IMPC)Tcp>,homozygote,c57bl/6ncrl,-0.774071,...,0.016955,0.025432,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5083.0,5198.0,2.0,3.0,5.0,TTGATC TGATCA GATCAT ATCATT TCATTG CATTGT ATTG...
3968,3968,body weight,body weight,mgi:6156505,MGI:1929266,Snd1,Snd1<em1(IMPC)Tcp>,homozygote,c57bl/6ncrl,-0.774071,...,0.016955,0.025432,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5083.0,5198.0,2.0,3.0,5.0,GGGTTA GGTTAC GTTACA TTACAG TACAGG ACAGGC CAGG...


In [28]:
len(train.gene_symbol.unique()) + len(val.gene_symbol.unique()) + len(test.gene_symbol.unique())

5499

In [19]:

model = AutoModelForSequenceClassification.from_pretrained('zhihan1996/DNA_bert_6', 
                                                           num_labels=1, 
                                                           ignore_mismatched_sizes=True).to("cuda")
wandb.init(project="DNA-Weight", entity="pcoady")
wandb_config = wandb.config

Some weights of the model checkpoint at zhihan1996/DNA_bert_6 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNA_bert_6 a

In [ ]:
mse_metric = evaluate.load("mse")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    #labels = labels.reshape(-1, 1)
    
    wandb.log({'True Values': labels })
    wandb.log({'Predicted Values': logits})

    mse = mse_metric.compute(predictions=logits, references=labels)
    return mse

In [ ]:
training_args = TrainingArguments(output_dir='weight_model', 
                                  evaluation_strategy='epoch',
                                  per_device_train_batch_size = 5,  
                                  num_train_epochs=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=val,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
results = trainer.predict(test)
fig = go.Figure()
fig.add_trace(go.Histogram(x=results[1], histnorm='probability', name = "actual"))
fig.add_trace(go.Histogram(x=results[0].reshape(-1, 1)[:,0], histnorm='probability', name = "predictions"))

fig.update_layout(
    barmode="overlay",
    bargap=0.1)


fig.show()